### Download 15 mins interval data from http://www.buildsmartdc.com/ using the HTTP REST API

##### API Format is
https://api.newcityenergy.com/v1/buildings/145/interval_data?start_time=1527566400&end_time=1530162000
##### This is an undocumented API, which is being used for ploting in their website

In [32]:
import os
import json
import requests
import string
from datetime import datetime
import pytz
import pandas as pd

In [33]:
raw_dir       = "../data/raw/dgs_15min_api/"
raw_dir1      = "../data/raw/"

#https://api.newcityenergy.com/v1/buildings/145/interval_data?start_time=1528516800&end_time=1531112400

API_FORMAT = 'https://api.newcityenergy.com/v1/buildings/{}/interval_data?start_time={}&end_time={}'
## another API - This will download all 15mins interval data
##API_FORMAT = 'https://api.newcityenergy.com/v1/buildings/{}/interval_data


if not os.path.exists(raw_dir):
    os.makedirs(raw_dir)

In [68]:
def download_DGS_intervaldata_(API, bid, epoch_start, epoch_end):
    url = API.format(bid, epoch_start, epoch_end)
    print "Downloading %3d - %s" % (bid, url)
    r = requests.get(url)
    if r.status_code == requests.codes.ok:
        return r.text.encode('ascii',errors='ignore')

In [69]:
def download_DGS_intervaldata(API, bid, epoch_start, epoch_end):
    isDone   = False
    res      = None
    attempts = 1    
    while(not isDone and attempts <= 5):
        try:
            res = download_DGS_intervaldata_(API, bid, epoch_start, epoch_end)
            isDone = True
        except Exception as e:
            attempts += 1
            print '\tException: %s, retrying - attempt %d' % (e, attempts)
    return res

In [70]:
def convert_to_df(res):
    js = json.loads(res)
    kWh       = js.values()    
    epoch     = map(float, js.keys())
    epoch     = map(int, epoch)
    #timestamp = pd.to_datetime(epoch, unit='s')
    timestamp = pd.to_datetime(epoch, unit='s').tz_localize('UTC').tz_convert('US/Eastern')
    
    df = pd.DataFrame({'epoch':epoch, 'timestamp':timestamp, 'kWh':kWh}).sort_values('epoch')    
    df.set_index('timestamp', inplace=True)
    return df

In [71]:
#epoch_start = 1388534400 # January 1, 2014 12:00:00 AM
#epoch_end   = 1514764799 # December 31, 2017 11:59:59 PM

epoch_start = 1325394000   # Human time (GMT): Sunday, January 1, 2012 5:00:00 AM
epoch_end   = 1530417600-1 # Human time (GMT): Sunday, July 1, 2018 5:00:00 AM

for bid in range(412,800):
    res = download_DGS_intervaldata(API_FORMAT, bid, epoch_start, epoch_end)    
    if( res == None):
        print '%3d No data found! ' % (bid)
        continue
    df  = convert_to_df(res)
    nrows = len(df)
    
    if(nrows < 4*24*365/2): # we need atleast six months of data
        print '%3d  %6d - Ignored ' % (bid, nrows)
    else:       
        filename = os.path.join(raw_dir, str(bid) + '.csv')
        print '%3d  %6d - %s' % (bid, nrows, filename)
        df.to_csv(filename)

412       0 - Ignored 
413  138597 - ../data/raw/dgs_15min_api/413.csv
414  107457 - ../data/raw/dgs_15min_api/414.csv
415  177937 - ../data/raw/dgs_15min_api/415.csv
416       0 - Ignored 
417       0 - Ignored 
418 No data found! 
419  189150 - ../data/raw/dgs_15min_api/419.csv
420  189357 - ../data/raw/dgs_15min_api/420.csv
421   91810 - ../data/raw/dgs_15min_api/421.csv
422       0 - Ignored 
423       2 - Ignored 
424       0 - Ignored 
425       0 - Ignored 
426  138693 - ../data/raw/dgs_15min_api/426.csv
427       0 - Ignored 
428       2 - Ignored 
429       2 - Ignored 
430  181460 - ../data/raw/dgs_15min_api/430.csv
431       0 - Ignored 
432 No data found! 
433 No data found! 
434  188800 - ../data/raw/dgs_15min_api/434.csv
435  138405 - ../data/raw/dgs_15min_api/435.csv
436       0 - Ignored 
437  179012 - ../data/raw/dgs_15min_api/437.csv
438   92598 - ../data/raw/dgs_15min_api/438.csv
439  189328 - ../data/raw/dgs_15min_api/439.csv
440       0 - Ignored 
441       0 - Ign

464  189328 - ../data/raw/dgs_15min_api/464.csv
465       2 - Ignored 
466   95405 - ../data/raw/dgs_15min_api/466.csv
467   96557 - ../data/raw/dgs_15min_api/467.csv
468  138497 - ../data/raw/dgs_15min_api/468.csv
469  181265 - ../data/raw/dgs_15min_api/469.csv
470  188940 - ../data/raw/dgs_15min_api/470.csv
471       0 - Ignored 
472       0 - Ignored 
473       0 - Ignored 
474  183910 - ../data/raw/dgs_15min_api/474.csv
475       2 - Ignored 
476  189374 - ../data/raw/dgs_15min_api/476.csv
477       2 - Ignored 
478  187625 - ../data/raw/dgs_15min_api/478.csv
479  188805 - ../data/raw/dgs_15min_api/479.csv
480  189349 - ../data/raw/dgs_15min_api/480.csv
481  189257 - ../data/raw/dgs_15min_api/481.csv
482  189177 - ../data/raw/dgs_15min_api/482.csv
483  189101 - ../data/raw/dgs_15min_api/483.csv
484   80226 - ../data/raw/dgs_15min_api/484.csv
485  184053 - ../data/raw/dgs_15min_api/485.csv
486       0 - Ignored 
487  189273 - ../data/raw/dgs_15min_api/487.csv
488  189087 - ../data/r

516       2 - Ignored 
517  189372 - ../data/raw/dgs_15min_api/517.csv
518  167321 - ../data/raw/dgs_15min_api/518.csv
519       0 - Ignored 
520  183317 - ../data/raw/dgs_15min_api/520.csv
521       0 - Ignored 
522  178109 - ../data/raw/dgs_15min_api/522.csv
523  188395 - ../data/raw/dgs_15min_api/523.csv
524  112930 - ../data/raw/dgs_15min_api/524.csv
525  189741 - ../data/raw/dgs_15min_api/525.csv
526       0 - Ignored 
527   29716 - ../data/raw/dgs_15min_api/527.csv
528       2 - Ignored 
529       0 - Ignored 
530  129389 - ../data/raw/dgs_15min_api/530.csv
531  136773 - ../data/raw/dgs_15min_api/531.csv
532       0 - Ignored 
533  189356 - ../data/raw/dgs_15min_api/533.csv
534   10561 - Ignored 
535  189766 - ../data/raw/dgs_15min_api/535.csv
536       0 - Ignored 
537       0 - Ignored 
538   13441 - Ignored 
539  220795 - ../data/raw/dgs_15min_api/539.csv
540   41082 - ../data/raw/dgs_15min_api/540.csv
541  189164 - ../data/raw/dgs_15min_api/541.csv
542       0 - Ignored 
543 

568  165505 - ../data/raw/dgs_15min_api/568.csv
569       0 - Ignored 
570  183693 - ../data/raw/dgs_15min_api/570.csv
571  183133 - ../data/raw/dgs_15min_api/571.csv
572  189166 - ../data/raw/dgs_15min_api/572.csv
573       0 - Ignored 
574  189132 - ../data/raw/dgs_15min_api/574.csv
575   53269 - ../data/raw/dgs_15min_api/575.csv
576  160537 - ../data/raw/dgs_15min_api/576.csv
577       0 - Ignored 
578  189081 - ../data/raw/dgs_15min_api/578.csv
579  189161 - ../data/raw/dgs_15min_api/579.csv
580  143043 - ../data/raw/dgs_15min_api/580.csv
581  189353 - ../data/raw/dgs_15min_api/581.csv
582       0 - Ignored 
583  186029 - ../data/raw/dgs_15min_api/583.csv
584  189245 - ../data/raw/dgs_15min_api/584.csv
585       2 - Ignored 
586       0 - Ignored 
587  189365 - ../data/raw/dgs_15min_api/587.csv
588       0 - Ignored 
589       2 - Ignored 
590  135521 - ../data/raw/dgs_15min_api/590.csv
591  180822 - ../data/raw/dgs_15min_api/591.csv
592       2 - Ignored 
593       0 - Ignored 
59

619       0 - Ignored 
620   98858 - ../data/raw/dgs_15min_api/620.csv
621       2 - Ignored 
622       0 - Ignored 
623  188996 - ../data/raw/dgs_15min_api/623.csv
624       0 - Ignored 
625       2 - Ignored 
626  189888 - ../data/raw/dgs_15min_api/626.csv
627   95601 - ../data/raw/dgs_15min_api/627.csv
628   95501 - ../data/raw/dgs_15min_api/628.csv
629   95501 - ../data/raw/dgs_15min_api/629.csv
630 No data found! 
631   95501 - ../data/raw/dgs_15min_api/631.csv
632       0 - Ignored 
633       0 - Ignored 
634       0 - Ignored 
635       0 - Ignored 
636   95601 - ../data/raw/dgs_15min_api/636.csv
637       0 - Ignored 
638       0 - Ignored 
639       0 - Ignored 
640       0 - Ignored 
641   75746 - ../data/raw/dgs_15min_api/641.csv
642       0 - Ignored 
643  189339 - ../data/raw/dgs_15min_api/643.csv
644  189612 - ../data/raw/dgs_15min_api/644.csv
645       0 - Ignored 
646       0 - Ignored 
647       0 - Ignored 
648       0 - Ignored 
649       0 - Ignored 
650       0 - I

675       0 - Ignored 
676   10655 - Ignored 
677 No data found! 
678 No data found! 
679 No data found! 
680 No data found! 
681 No data found! 
682 No data found! 
683 No data found! 
684 No data found! 
685 No data found! 
686 No data found! 
687 No data found! 
688 No data found! 
689 No data found! 
690 No data found! 
691 No data found! 
692 No data found! 
693 No data found! 
694 No data found! 
695 No data found! 
696 No data found! 
697 No data found! 
698 No data found! 
699 No data found! 
700 No data found! 
701 No data found! 
702 No data found! 
703 No data found! 
704 No data found! 
705 No data found! 
706 No data found! 
707 No data found! 
708 No data found! 
709 No data found! 
710 No data found! 
711 No data found! 
712 No data found! 
713 No data found! 
714 No data found! 
715 No data found! 
716 No data found! 
717 No data found! 
718 No data found! 
719 No data found! 
720 No data found! 
721 No data found! 
722 No data found! 
723 No data found! 
724 No data fo

734 No data found! 
735 No data found! 
736 No data found! 
737 No data found! 
738 No data found! 
739 No data found! 
740 No data found! 
741 No data found! 
742 No data found! 
743 No data found! 
744 No data found! 
745 No data found! 
746 No data found! 
747 No data found! 
748 No data found! 
749 No data found! 
750 No data found! 
751 No data found! 
752 No data found! 
753 No data found! 
754 No data found! 
755 No data found! 
756 No data found! 
757 No data found! 
758 No data found! 
759 No data found! 
760 No data found! 
761 No data found! 
762 No data found! 
763 No data found! 
764 No data found! 
765 No data found! 
766 No data found! 
767 No data found! 
768 No data found! 
769 No data found! 
770 No data found! 
771 No data found! 
772 No data found! 
773 No data found! 
774 No data found! 
775 No data found! 
776 No data found! 
777 No data found! 
778 No data found! 
779 No data found! 
780 No data found! 
781 No data found! 
782 No data found! 
783 No data found! 


793 No data found! 
794 No data found! 
795 No data found! 
796 No data found! 
797 No data found! 
798 No data found! 
799 No data found! 


In [52]:
print type(res)
isinstance(res, Exception)

<class 'requests.exceptions.ConnectionError'>


True

In [ ]:
#test
epoch_start = 1388552400   # Human time (GMT): Wednesday, January 1, 2014 5:00:00 AM
epoch_end   = 1530417600-1 # Human time (GMT): Sunday, July 1, 2018 5:00:00 AM
bid = 1
res = download_DGS_intervaldata(API_FORMAT, bid, epoch_start, epoch_end)
print type(res), len(res)
#filename = os.path.join(raw_dir, str(bid) + '.json')
#with open(filename, 'wb') as fd:
#    fd.write(res)

In [ ]:
resJs = json.loads(res)

js = json.loads(res)
kWh       = js.values()    
epoch     = map(float, js.keys())
epoch     = map(int, epoch)
timestamp = pd.to_datetime(epoch, unit='s').tz_localize('UTC').tz_convert('US/Eastern')
#timestamp = pd.to_datetime(epoch, unit='s').tz_localize('US/Eastern', ambiguous='infer')

    
df = pd.DataFrame({'epoch':epoch, 'timestamp':timestamp, 'kWh':kWh}).sort_values('epoch')    
df.set_index('timestamp', inplace=True)

filename = os.path.join(raw_dir1, str(bid) + '.csv')
df.to_csv(filename)

#df['epoch'] = pd.to_numeric(df['epoch'])
#pd.to_datetime([1349720105, 1349806505, 1349892905,unit='s')
print type(resJs.keys()[0]), resJs.keys()[0]
print type(timestamp)
print type(kWh)
print df.shape
df.head()

In [ ]:
for tz in pytz.all_timezones:
    print tz